In [8]:
import sqlite3
import os, sys
import pandas as pd
from datetime import datetime 
import subprocess

#data_dir = '/Users/xlx/Downloads/graph-data'
data_dir = '/home/xlx/d2/MicrosoftAcademicGraph'
db_file = os.path.join(data_dir, 'AcademicGraph.db')

## data exploration with AcademicGraph

* citation to/from a certain conference/journal

* inter-disciplinary authors (??)

* paper count and citation count over time for a given FoS

#### Task 1 citations to/from a certain conference 

required databases:

* PaperReferences
* Papers
* ConferenceSeries

- test step 1: take one conference, in one year, find all the from/to citation

In [2]:
paper_ref_file = os.path.join(data_dir, 'data_txt', 'PaperReferences.txt')

with open(paper_ref_file, 'rt') as fh:
    a = fh.read(100000).split('\n')
    b = [s.split('\t') for s in a]
    print( sys.getsizeof(a) ) 
    print( sys.getsizeof(b) ) 
b[:5], len(b)    

45960
48464


([['000000A3', '619CE22C'],
  ['000000A3', '6CD617A9'],
  ['0000011D', '6B5946B4'],
  ['0000013A', '588A4926'],
  ['000001F8', '5B521D2E']],
 5556)

In [3]:
print( '{} start reading {} ... '.format(datetime.now(), paper_ref_file))

#with open(paper_ref_file, 'rt') as fh:
#    paper_ref = fh.read().split('\n')
#print( '{} load ref db of {} records, {} bytes'.format(datetime.now(), len(paper_ref), sys.getsizeof(paper_ref)) ) 
"""
2016-01-22 11:16:33.806514 start reading /home/xlx/d2/MicrosoftAcademicGraph/data_txt/PaperReferences.txt ... 
2016-01-22 11:21:54.427725 load ref db of 952364265 records, 7644136040 bytes
"""

with open(paper_ref_file, 'rt') as fh:
    paper_ref_buf = fh.read()
print( '{} load ref db {} bytes'.format(datetime.now(), sys.getsizeof(paper_ref_buf)) ) 

2016-01-22 13:53:13.483420 start reading /home/xlx/d2/MicrosoftAcademicGraph/data_txt/PaperReferences.txt ... 
2016-01-22 13:54:36.357696 load ref db 17142556801 bytes


In [32]:
conf_file = os.path.join(data_dir, 'data_txt', 'ConferenceSeries.txt')
conf_df = pd.read_table(conf_file, header=None, names=['ConfID', 'Abbrv', 'FullName'])

conf_list = ['PLDI', 'MM', 'CVPR', 'NIPS', 'ICML', 'IJCAI']

In [33]:
for c in conf_list:
    row = conf_df.loc[conf_df['Abbrv'] == c]
    conf_id = list(row['ConfID'])[0]
    #conf_id = '43AA5802' # "43AA5802	MM	ACM Multimedia"
    conf_paper_file = os.path.join(data_dir, 'papers.'+ c +'.txt')
    print( conf_id)
    print(conf_paper_file)
    
    if not os.path.exists(conf_paper_file):
        paper_file = os.path.join(data_dir, 'data_txt', 'Papers.txt')
        cmd = "grep " + conf_id +' '+ paper_file + ' > ' + conf_paper_file
        print("{} {}".format(datetime.now(), cmd) )
        subprocess.call(cmd)

    df_paper = pd.read_table(conf_paper_file, header=None, 
                             names=['PaperID', 'TitleOrig', 'TitleNorm', 'PubYear', 'PubDate', 
                                   'DOI', 'VenueOrig', 'VenueNorm', 'JournalID', 'ConfID', 'PaperRank' ])
    df_paper.head()
    set_paper = set(df_paper['PaperID'])

    print('{} papers in conference {} - {}'.format(len(set_paper), conf_id, c) )
    
    # init output vars/files
    citing_records = list()
    cited_records = list()
    citing_cnt = 0 
    cited_cnt = 0 
    ptr = 0 
    line_cnt = 0
    while ptr < len(paper_ref_buf):
    #for i in range(len(paper_ref)):
        #  each line: ['PaperID', 'RefID']
        eol = paper_ref_buf.find('\n', ptr)
        tmp = paper_ref_buf[ptr:eol].split('\t')
        line_cnt += 1
        ptr = eol + 1

        if tmp[0] in set_paper:
            citing_records.append(tmp)
            citing_cnt += 1
        if tmp[1] in set_paper:
            cited_records.append(tmp)
            cited_cnt += 1

        if line_cnt%10000000 == 0 :
            print('{} {:9.0f} lines, found {:6.0f} citing, {:6.0f} cited'.format(datetime.now(), line_cnt, citing_cnt, cited_cnt) )
            if line_cnt >= 1e9:
                break

    print('{} {:9.0f} lines, found {:6.0f} citing, {:6.0f} cited'.format(datetime.now(), line_cnt, citing_cnt, cited_cnt) )

    citing_file = os.path.join(data_dir, 'citing.'+c+'.txt')
    with open(citing_file, 'wt') as fh:
        fh.write('\n'.join(['\t'.join(s) for s in citing_records] ))

    cited_file = os.path.join(data_dir, 'cited.'+c+'.txt')
    with open(cited_file, 'wt') as fh:
        fh.write('\n'.join(['\t'.join(s) for s in cited_records] ))

    print(' saved to {} and {} \n\n'.format(citing_file, cited_file))

4332078E
/home/xlx/d2/MicrosoftAcademicGraph/papers.PLDI.txt
1091 papers in conference 4332078E - PLDI
2016-01-22 14:21:21.890580  10000000 lines, found    197 citing,    634 cited
2016-01-22 14:21:42.485302  20000000 lines, found    559 citing,   1334 cited
2016-01-22 14:22:03.049098  30000000 lines, found    931 citing,   2039 cited
2016-01-22 14:22:23.568270  40000000 lines, found   1301 citing,   2762 cited
2016-01-22 14:22:44.091638  50000000 lines, found   1533 citing,   3400 cited
2016-01-22 14:23:04.593455  60000000 lines, found   1847 citing,   4077 cited
2016-01-22 14:23:25.719149  70000000 lines, found   2224 citing,   4755 cited
2016-01-22 14:23:46.847716  80000000 lines, found   2539 citing,   5491 cited
2016-01-22 14:24:07.967250  90000000 lines, found   2806 citing,   6127 cited
2016-01-22 14:24:29.069634 100000000 lines, found   3137 citing,   6777 cited
2016-01-22 14:24:50.201901 110000000 lines, found   3502 citing,   7537 cited
2016-01-22 14:25:11.341473 120000000 li

In [27]:
list(row['ConfID'])[0]

'4332078E'

In [ ]:
with open(paper_ref_file, 'rt') as fh:
    df_paperref = pd.read_table(fh.read(50000000), header=None, 
                         names=['PaperID', 'RefID']) 
    print( sys.getsizeof(df_paperref) ) 
    df_paperref.head()

In [ ]:


"""
conn = sqlite3.connect(db_file)
c = conn.cursor()

cmd = "SELECT * FROM Papers WHERE Paper_publish_year='{}' AND Conference_series_ID_mapped_to_venue_name='{}'".format(year, conf_id)
#c.excute(cmd)
for row in c.execute(cmd):
    print row

conn.close()
"""